# TP1 using API on Camille

Imports

In [21]:
import os
import json
import requests
import pandas as pd
import pdfplumber

Préparration des variables pour utiliser l'API NLP Cloud 

In [ ]:
# configuration NLPCloud 
API_KEY = "d7ce4ca1b17e8e266697a80329b307cc3a804079"
headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

Utilisation des fichiers téléchargés

In [27]:
path = "../../data/tp1/articles"

files = os.listdir(path)

print(files)

['KB_JB230_1892-08-07_01-0003.pdf', 'KB_JB230_1903-10-16_01-0002.pdf', 'KB_JB230_1913-07-05_01-0001.pdf', 'KB_JB258_1884-09-03_01-0003.pdf', 'KB_JB258_1894-12-09_01-0003.pdf', 'KB_JB258_1906-01-09_01-0002.pdf', 'KB_JB421_1899-05-15_01-00003.pdf', 'KB_JB421_1926-10-29_01-00002.pdf', 'KB_JB421_1950-04-15_01-00004.pdf', 'KB_JB427_1920-01-10_01-00004.pdf', 'KB_JB427_1933-01-04_01-00003.pdf', 'KB_JB427_1949-07-18_01-00008.pdf', 'KB_JB449_1846-05-30_01-00002.pdf', 'KB_JB449_1912-01-04_01-00003.pdf', 'KB_JB449_1947-08-29_01-00003.pdf', 'KB_JB494_1853-10-30_01-0002.pdf', 'KB_JB494_1922-09-28_01-0005.pdf', 'KB_JB494_1939-12-08_01-0004.pdf', 'KB_JB555_1836-02-08_01-00002.pdf', 'KB_JB555_1899-01-19_01-00003.pdf', 'KB_JB555_1940-03-01_01-00004.pdf', 'KB_JB567_1857-02-02_01-00003.pdf', 'KB_JB567_1892-01-03_01-00005.pdf', 'KB_JB567_1924-08-30_01-00003.pdf', 'KB_JB572_1850-03-15_01-00003.pdf', 'KB_JB572_1927-07-20_01-00005.pdf', 'KB_JB572_1950-06-07_01-00004.pdf', 'KB_JB638_1860-05-21_01-00002.pdf', 

Analyse

In [28]:
if len(files) > 0:
    # Extraction du texte du premier PDF
    pdf_path = os.path.join(path, files[0])
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            content = page.extract_text()
            if content:
                text += content + "\n"
                
print (text)

■ Mi-.......nèfoiifoTAMM. m
13.50; 4 derniers mois, 15.70: i moto de novembre, Première installation da
1. — A relut, courant, I3JRH aaptemfere, 15.78»
SYSTÈME KNEIPP
'sidt m!» à IB0O«' * m0fa ** ** «*-
JÊ%i î&z tsssix.
» dans 1* prou. Rhénane et t Allemagne du Kord {“l'v quai.4 la fabric
O ra é is .W in* è n O o O i . r T s O . ; p 1. o 8 i 0 l. 4 0 . 4 » .0 4 0 ) 9 M .40 . ; fc p t é a c n h m é , * , t 1 t .5 è 8 f 4 i 3 1 J .6 D *; ; - ta n c / Bêb forum : MMm la Ssf.ità Colope PETITAT
tomate*, 0.50 4 9.80. m Sacres (raouuxx 8888»«., 3316 .45/00 00) ld. téaoes a* t. août, Recommandée par plusieun lommiléa médicales bel Si, rut Rlaneittssene.
Poissons.— Bcrevlaaet, le cent. 8.06410.08; hnltrm, 37.00) septembre, 83'7.00; 4 moto d’octobre, 36.48; ges et aUemaades' Site charmant, contrée montagneuse Fond«*t» ! Koekelbtrg-
t-M 4 3.80; homards, la ptoca» «.08 4 US. ■ ïafflta’fa'priifiv et boisée, confert satisfaisant toutes exigences, cha­ W Rovoi eatalcgue franco
Aloat, g août. - BouMou

Le OCR est mauvais, mais peu importe on va quand même tester l'API

In [ ]:
# Limiter le texte à 500 caractères pour l'API (gratuit)
text = text[:500]

# utilisation de l'API NLP Cloud pour l'extraction des entités nommées
url = "https://api.nlpcloud.io/v1/fr_core_news_lg/entities"

# Initialisation de la liste pour stocker les résultats
data = []

# Préparation de la requête
payload = {"text": text}
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    entities = response.json().get("entities", [])
    for ent in entities:
        data.append({
            "entite": ent["text"],
            "type": ent["type"]
        })
    print(f"{len(entities)} entités extraites.")
    print (data)
else:
    print(f"Erreur {response.status_code}: {response.text}")


13 entités extraites.
[{'entite': 'Première installation da\n1', 'type': 'MISC'}, {'entite': 'I3JRH aaptemfere', 'type': 'MISC'}, {'entite': 'IB0O', 'type': 'MISC'}, {'entite': 'Rhénane', 'type': 'LOC'}, {'entite': 'Allemagne', 'type': 'LOC'}, {'entite': "Kord {“l'v quai.4", 'type': 'MISC'}, {'entite': 'fabric\nO ra é is .W', 'type': 'LOC'}, {'entite': 'O o O i', 'type': 'LOC'}, {'entite': 'r T s O', 'type': 'MISC'}, {'entite': 'fc p t é', 'type': 'PER'}, {'entite': 'J .6', 'type': 'MISC'}, {'entite': 'Bêb', 'type': 'MISC'}, {'entite': 'MMm', 'type': 'MISC'}]


Les données sont peu cohérente mais cela est dû à la mauvaise qualité de l'input (à cause de l'OCR)